In [3]:
#Set environment options
import pandas as pd
# settings to display all columns
pd.set_option("display.max_columns", None)

In [2]:
#function to combine matchup data
def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        #pass
        return None
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result

In [5]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import teams
import time

#get nba team data including id's
nba_teams = pd.DataFrame(teams.get_teams())
#Create list of NBA team id's
team_ids = nba_teams['id'].tolist()

#season data should be pulled for 
s = '2022-23'


#function to call api and get all NBA games for a given season
def get_all_games(team__ids,season):
    team_season = []
    for i in team_ids:
        games = leaguegamefinder.LeagueGameFinder(team_id_nullable = i,season_nullable = season,season_type_nullable ='Regular Season').get_data_frames()[0]
        team_season.append(games)
        nba_season = pd.concat(team_season)
        time.sleep(1)
    return(nba_season)

nba_season = get_all_games(team_ids,s)
nba_season = nba_season.sort_values(by=['GAME_DATE','GAME_ID'])

In [6]:
home_team = nba_season[nba_season.MATCHUP.str.contains(' vs. ')]
away_team = nba_season[nba_season.MATCHUP.str.contains(' @ ')]

In [11]:
home_team_1 = home_team[['SEASON_ID','GAME_ID','GAME_DATE','TEAM_ID','TEAM_ABBREVIATION','MATCHUP','WL']]
home_team_1= pd.get_dummies(home_team_1, columns=["TEAM_ABBREVIATION"], prefix=["HOME"] )

In [24]:
from nba_api.stats.endpoints import teamdashboardbyopponent

call = teamdashboardbyopponent.TeamDashboardByOpponent(team_id=1610612742,last_n_games=5)#,date_to_nullable='2022-10-19'
overall_team_dashboard = call.get_data_frames()[0]
#conference_team_dashboard = call.get_data_frames()[1]
#division_team_dashboard = call.get_data_frames()[2]
opponent_team_dashboard = call.get_data_frames()[3]

In [9]:
dum_df = pd.get_dummies(home_team_1, columns=["TEAM_ABBREVIATION"], prefix=["HOME"] )



In [10]:
dum_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 220 entries, 14 to 0
Data columns (total 36 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   SEASON_ID  220 non-null    object
 1   GAME_ID    220 non-null    object
 2   GAME_DATE  220 non-null    object
 3   TEAM_ID    220 non-null    int64 
 4   MATCHUP    220 non-null    object
 5   WL         220 non-null    object
 6   HOME_ATL   220 non-null    uint8 
 7   HOME_BKN   220 non-null    uint8 
 8   HOME_BOS   220 non-null    uint8 
 9   HOME_CHA   220 non-null    uint8 
 10  HOME_CHI   220 non-null    uint8 
 11  HOME_CLE   220 non-null    uint8 
 12  HOME_DAL   220 non-null    uint8 
 13  HOME_DEN   220 non-null    uint8 
 14  HOME_DET   220 non-null    uint8 
 15  HOME_GSW   220 non-null    uint8 
 16  HOME_HOU   220 non-null    uint8 
 17  HOME_IND   220 non-null    uint8 
 18  HOME_LAC   220 non-null    uint8 
 19  HOME_LAL   220 non-null    uint8 
 20  HOME_MEM   220 non-null    uint8 